## Реализация алгоритма определения маркетплейса из поисковой выдачи


### Краткое описание
В процессе решения задачи поиска поставщиков, возникла необходимость индитификации и разделения уникальных поставщиков и т.н. маркетплейсов (агрегаторов) в результатах поисковой выдачи. Для этого был произведен поиск действующих маркетплейсов с целью получения метаданных главной страницы. 

Проанализированные сайты:

In [1]:
# Добавляем метаданные известных маркетплейсов
# в датафрейм для последующего анализа
import requests
import bs4 as bs
import pandas as pd

headers = {'user-agent': "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15",
           'accept-encoding': 'gzip, deflate, br',
           'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
           'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           'connection': 'keep-alive',
           'Sec-Fetch-Dest': 'document',
           'Sec-Fetch-Mode': 'navigate',
           'Sec-Fetch-Site': 'none',
           'Sec-Fetch-User': '?1',
           'Upgrade-Insecure-Requests': '1'}

#TODO:Ra3Svet добавить в словарь маркетплейсы
URLS = {
    "Металлопрокат": "https://www.metalloprokat.ru",
    "Ozon": "https://www.ozon.ru",
    "Yandex.Market": "https://market.yandex.ru",
    "Сбер МегаМаркет": "https://sbermegamarket.ru",
    "Wildberries": "https://www.wildberries.ru"
}

def getMetaDescrDF(URLS):

    marketplaces = pd.DataFrame(columns=['Наименование', 'URL', 'Описание'])

    for element in URLS.keys():
        headers['host'] = str(URLS[element])[8:]
        req = requests.get(URLS[element], headers=headers)
        if req.status_code != 200:
            print(f"{URLS[element]} -> {req.status_code}")
            continue
        req.encoding = 'utf-8'
        parsed = bs.BeautifulSoup(req.text, 'lxml')
        meta = parsed.find_all('meta', {'name': 'description'})
        for m in meta:
            marketplaces = marketplaces.append({'Наименование': str(element), 'URL': str(
                URLS[element]), 'Описание': str(m['content'])}, ignore_index=True)
    
    return marketplaces

marketplaces = getMetaDescrDF(URLS)
marketplaces

https://market.yandex.ru -> 403


,Наименование,URL,Описание
0,Металлопрокат,https://www.metalloprokat.ru,Купить выгодно металлопрокат оптом и в розницу...
1,Сбер МегаМаркет,https://sbermegamarket.ru,Маркетплейс sbermegamarket.ru - это место выго...
2,Wildberries,https://www.wildberries.ru,"Коллекции женской, мужской и детской одежды, о..."


In [2]:
# подготовка метаданных к анализу
import re
from pymorphy2 import MorphAnalyzer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from string import punctuation

tmplNotLetters = re.compile(r'[^a-zA-ZА-Яа-я\s]{1,}')

removable_symbols = [word for word in stopwords.words('russian')] + list(punctuation) 

def lemmatizeText(text):
    notLetters = re.findall(tmplNotLetters, text)
    result = text
    for notLetter in notLetters:
        result = result.replace(notLetter, "").lower()
   # no_punct_text = ''.join([s for s in result if s not in punctuation])
    clear_text = [word for word in result.split() if word not in removable_symbols]
    
    result = ''
    for word in clear_text:
        result = result+' '+MorphAnalyzer().parse(word)[0].normal_form

    return result

marketplaces['Нормализированное описание'] = None
for i in range(len(marketplaces['Описание'])):
   marketplaces['Нормализированное описание'][i] = lemmatizeText(marketplaces['Описание'][i])

marketplaces

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Наименование,URL,Описание,Нормализированное описание
0,Металлопрокат,https://www.metalloprokat.ru,Купить выгодно металлопрокат оптом и в розницу...,купить выгодно металлопрокат оптом розница ро...
1,Сбер МегаМаркет,https://sbermegamarket.ru,Маркетплейс sbermegamarket.ru - это место выго...,маркетплейс sbermegamarketru это место выгодн...
2,Wildberries,https://www.wildberries.ru,"Коллекции женской, мужской и детской одежды, о...",коллекция женский мужской детский одежда обув...


In [5]:
# определяем объединение множеств слов всех описаний для формирования "ядра"
proto_core = set.union(*[set(s.split())for s in marketplaces['Нормализированное описание']])
proto_core

{'metalloprokat',
 'sbermegamarketru',
 'актуальный',
 'база',
 'больший',
 'вещь',
 'выбор',
 'выбрать',
 'выгодно',
 'выгодный',
 'гарантия',
 'детский',
 'дом',
 'доставка',
 'женский',
 'интернетмагазин',
 'информация',
 'коллекция',
 'компания',
 'контакт',
 'купить',
 'маркетплейс',
 'место',
 'металлопрокат',
 'мужской',
 'обувь',
 'одежда',
 'оплата',
 'оптом',
 'покупка',
 'поставщик',
 'предлагать',
 'прямой',
 'размер',
 'реальный',
 'розница',
 'россия',
 'самостоятельно',
 'сбермегамаркет',
 'сервис',
 'совет',
 'спорт',
 'сравнивать',
 'таблица',
 'также',
 'технология',
 'товар',
 'удобный',
 'уход',
 'цена',
 'это'}